######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
#sys.path.append(os.path.abspath("../azure-enterprise-scale-ml/esml/common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
#p.dev_test_prod="dev"
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

######  NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLDataset, ESMLProject

p = ESMLProject()
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
######  NB!

## Always run thic CELL below
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPieplinefactory

In [1]:
import sys
sys.path.insert(0, "../../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = True
p.active_model = 10 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds01_titanic.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds02_haircolor.py
File to EDIT (step: IN_2_SILVER_3): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds03_housing.py
File to EDIT (step: IN_2_SILVER_4): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds04_lightsaber.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M10/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M10/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M10/train_split_and_register.py
File to EDIT (step: TRAIN_MANUA

# IN_2_GOLD_SCORING
- Full ML-workflow: If you want to refine data from IN to GOLD, and SCORE model on GOLD, saves SCORED_GOLD in datalake
- Scenario: You want MLOps and full automation, ESMLPipelineFactory starting from Azure Data factory, and calling this genereated Azure ML Pipeline. 
    - Pipeline saving data automatically using the enterprise datalake/ESML AutoLake and ESML SDK

In [2]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING)

Creates template step_files.py for user to edit at:
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds01_titanic.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds02_haircolor.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds03_housing.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds04_lightsaber.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/silver_merged_2_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/scoring_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/train_split_and_register.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/train_manual.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M10/your_code/your_custom_code.py
Using GEN2 as Datastore
use_project_sp_2_mount: True
ESML will auto-create a compute...
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> ove

In [3]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

execute_pipeline (scoring): Inference_mode: 1
Created step IN 2 SILVER - ds01_titanic [69623d05][af892f67-d623-4092-9c2e-4badff16681e], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_haircolor [74fe300f][cd6abae9-46ec-4f30-b528-563b28ad96b9], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds03_housing [4165ba4d][2055303f-89a5-4350-aeb1-e3174e935274], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds04_lightsaber [c7f52d28][c3fd9872-d2e6-4256-b62d-fac03b385d12], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [b682e94e][8f938e7d-33a2-4483-8720-c262818d3446], (This step will run and generate new outputs)
Created step SCORING GOLD [6669b9b9][8687452a-5f3f-4eee-b045-ecf85ed0e2bc], (This step will run and generate new outputs)
Submitted PipelineRun 53765ba4-8197-465c-969e-09f795c2c74f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/53765ba4-8197-465c-969e-09f79

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "See child run or execution logs for more details.",
        "messageFormat": "Child run failed for UserError",
        "messageParameters": {},
        "details": []
    },
    "environment": "westeurope",
    "location": "westeurope",
    "time": "2022-06-23T00:04:22.248249Z",
    "componentName": ""
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"See child run or execution logs for more details.\",\n        \"messageFormat\": \"Child run failed for UserError\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"environment\": \"westeurope\",\n    \"location\": \"westeurope\",\n    \"time\": \"2022-06-23T00:04:22.248249Z\",\n    \"componentName\": \"\"\n}"
    }
}

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

# IN_2_GOLD  only (for scoring/inference purpose)
- If just wanting to refine data to GOLD, before SCORE step
- Scenario: You want to refine data from "IN_2_GOLD" with an easy way to READ/WRITE data (using the enterprise datalake via ESML AutoLake and ESML SDK)

In [2]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

Creates template step_files.py for user to edit at:
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/your_code/your_custom_code.py
Using GEN2 as Datastore
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing

In [3]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Created step IN 2 SILVER - ds01_diabetes [566ec0b8][b2d540ba-a55d-4faa-8414-314a9a323d45], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_other [508287f7][369d0e83-5347-4aa0-8ded-130d69ed447b], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [c2384977][a02d9e29-08df-4655-86cb-36bd2abe9df3], (This step will run and generate new outputs)
Submitted PipelineRun c64f3e04-10ba-4d4c-a82b-0cf51a6c8115
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c64f3e04-10ba-4d4c-a82b-0cf51a6c8115?wsid=/subscriptions/ca0a8c40-b06a-4e4e-8434-63c03a1dee34/resourcegroups/MSFT-WEU-EAP_PROJECT02_AI-DEV-RG/workspaces/msft-weu-DEV-eap-proj02_ai-amls&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline submitted for execution!
PipelineRunId: c64f3e04-10ba-4d4c-a82b-0cf51a6c8115
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/c64f3e04-10ba-4d4c-a82b-0cf51a6c8115?wsid=/subscriptions/ca0a8c40-b06a-4e4e-8434-63c03a1dee34/reso

'Finished'

In [4]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline) # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_6
pub_pipe.id 5ecadc66-0895-4633-b7de-2fd3df5812ce
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_5
pub_pipe.id 5d828765-e496-4df6-9701-4581c31990e6
pub_pipe.name 11_diabetes_model_reg_batch_scoring_pipe_EP_4
pub_pipe.id e5ed3b30-89fd-4b70-a2a0-403451ea2228
pub_pipe.name 11_diabetes_model_reg_pipe_IN_2_GOLD
pub_pipe.id 5c1c0622-2f18-4380-865f-b5df380f2345
pub_pipe.name 10_titanic_model_clas_batch_scoring_pipe_EP_4
pub_pipe.id aa67ac38-da18-4b2d-973a-df87998aa1b6
pub_pipe.name 11_diabetes_model_reg_batch_scoring_pipe_EP_1
pub_pipe.id c21397ba-8e26-4582-b7eb-0cf9bee5cc0c


# IN_2_GOLD only - TRAIN...or refined data for Power BI
- If just wanting to refine data to GOLD, before TRAIN-step, `or just to prep data for a Power BI report (No ML involved)`
- Scenario: You want to refine data from "IN_2_GOLD" with an easy way to READ/WRITE data (using the enterprise datalake via ESML AutoLake and ESML SDK)

In [3]:
p.inference_mode = True
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
Searching for setting in ESML datalake...
ESML in-folder settings override = TRUE 
 - Found settings in the ESML AutoLake  [active_in_folder.json,active_scoring_in_folder.json], to override ArgParse/GIT config with.
 - TRAIN in date:  2021/01/01
 - INFERENCE in date: 2021/06/08 and ModelVersion to score with: 1 (0=latest)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster prj02-m11-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = prj02-m11-dev


'Finished'

# Change environment 
- Other curated
- Non-curated, CONDA AutoML
- https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments

In [1]:
import sys
sys.path.insert(0, "../../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = True
p.active_model = 11 # Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_split_and_register.py
File to EDIT (step: TRAIN_MANUAL): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_manual.py
File to EDIT a lot (reference in step-scripts Custom code): ../../../2_A_aml_pipeline/4_inference/batch/M11/your_code/your_custom_code.py

 ---- WHAT model to

## Other curated - change environment

In [ ]:

p_factory.use_curated_automl_environment = True
p_factory.environment_name = "AzureML-AutoML-DNN" # Training[ "AzureML-AutoML", "AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]  Inference["AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference",]

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING,same_compute_for_all=True)

## Non-curated - custom conda/pip definition

In [3]:
p_factory.use_curated_automl_environment = False

######### See here for environments: https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments

######### ESML Defaults to the below CONDA, when use_curated_automl_environment = False 
#aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    #conda_packages=['pandas','scikit-learn'], 
    #pip_packages=['azureml-sdk[automl]', 'pyarrow'])

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING,same_compute_for_all=True)

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
ESML will auto-create a compute...
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster p02-m11weu-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = p02-m11weu-dev
Reusing existing compute...
create_gold_to_score_step: inference_mode=True


In [3]:
# RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

execute_pipeline (scoring): Inference_mode: 1
Created step IN 2 SILVER - ds01_diabetes [c605edac][e1a023cd-087f-4846-b028-f1c745e9c1fa], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_other [21b747c9][bc5c6955-2e35-4deb-8d88-da06979d085c], (This step will run and generate new outputs)
Created step SILVER MERGED 2 GOLD [16d53dee][d342e946-4fb9-45f5-9149-e37f389b447c], (This step will run and generate new outputs)
Created step SCORING GOLD [ad3110fe][c47a336f-fb4b-459a-80ea-67d73df1a553], (This step will run and generate new outputs)
Submitted PipelineRun 1aa136d7-d109-405b-ba35-677a32c10919
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1aa136d7-d109-405b-ba35-677a32c10919?wsid=/subscriptions/ca0a8c40-b06a-4e4e-8434-63c03a1dee34/resourcegroups/MSFT-WEU-EAP_PROJECT02_AI-DEV-RG/workspaces/msft-weu-DEV-eap-proj02_ai-amls&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline submitted for execution!
PipelineRunId: 1aa136d7-d109-405b-ba35-677a32c1

'Finished'

# Edit/Customize the ESML auto-generated pipeline

In [ ]:
from azureml.pipeline.core import Pipeline

#1) Get the ESML auto-generated steps
step_array = p_factory.pipeline_steps_array

for(step in step_array):
    t1 = type(step)
    print(str(t1))

#2)  ...edit as you wish...change compute, environments, etc...

pipeline = Pipeline(workspace = p.ws, steps=step_array) # 3) Create a pipelin

#4) RUN the pipeline as
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

# GOLD_SCORING
- If just wanting to SCORE, with best trained model in Azure ML Studio
- Scenario: Another service, Databricks or Azyre Synapse, refined data fron IN_2_GOLD", you just want to use Azure ML for scoring
    - Azure ML benefits:
        - Get lineage via Azure ML Datasets and Azure ML Model linage. 
        - Host the batch scoring pipleine in your Azure ML Studio
    - EMSL benefits
        - ESMLPipelinefactory + ESML AutoLake design, ESML extra logging & lineage.



In [ ]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.GOLD_SCORING) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

## 1b) When satisfied - `PUBLISH` pipeline (or rebuild and publish)

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline, "_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint